# Random Forest Algorithm

#### Importación de librerías ⬇️

In [2]:
import pickle
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import export_text, plot_tree
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor  # Asegúrate de importar DecisionTreeRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

C:\Users\danar\AppData\Local\Temp\ipykernel_28716\3307797748.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Importar data de un csv
data = pd.read_csv('los_data_gt.csv')

In [14]:
def regressionRandomForest(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30]
    }

    # Crear el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=42)

    # Crear el objeto GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    # Entrenar el modelo
    grid_search.fit(X_train, y_train)

    # Obtener los mejores parámetros y el mejor estimador
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Entrenar el mejor modelo en todo el conjunto de entrenamiento
    best_estimator.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = best_estimator.predict(X_test)

    # Calcular el error cuadrático medio
    mse = mean_squared_error(y_test, y_pred)
    print("Error cuadrático medio:", mse)

    # Calcular la raiz del error cuadratico medio
    rmse = root_mean_squared_error(y_test, y_pred)
    print("raiz del error cuadratico medio", rmse)

    # Calcular el coeficiente de determinación
    r2 = r2_score(y_test, y_pred)
    print("Coeficiente de determinación:", r2)





In [8]:
data.columns

Index(['dias_estancia', 'sexo', 'grupo_etnico', 'departamento', 'municipio',
       'causa_atencion', 'condicion_egreso', 'tratamiento_recibido',
       'edad_correcta', 'causa_categoria', 'edad_categoria', 'region'],
      dtype='object')

In [5]:
data.head()

,dias_estancia,sexo,grupo_etnico,departamento,municipio,causa_atencion,condicion_egreso,tratamiento_recibido,edad_correcta,causa_categoria,edad_categoria,region
0,1.0,Hombre,Ignorado,Guatemala,San Juan Sacatepéquez,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Central
1,1.0,Mujer,Ignorado,Zacapa,Zacapa,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Nororiente
2,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente
3,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente
4,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente


In [18]:
dr = data.copy()

In [19]:
# Evaluación de variables
def change(variable, dictValue, df2):
    count = 1

    def changeToVariable(x):
        nonlocal count
        nonlocal dictValue
        if x in dictValue:
            return dictValue[x]
        else:
            dictValue[x] = count
            count += 1
            return dictValue[x]
        
    if not pd.api.types.is_numeric_dtype(df2[variable]):
        df2[variable] = df2[variable].apply(changeToVariable)
    
        print(f'Para la variable {variable} los valores ahora son: ')
        for key, value in dictValue.items():
            print(key, ":", value)
    else:
        print(f"La columna {variable} es numerica")


def chageNotNumericVars(df):
    dicVars = {}
    for col in df.columns:
        change(col, dicVars, df)
    return dicVars

In [ ]:
y = dr.pop('dias_estancia')
x = dr
description = chageNotNumericVars(dr)

In [20]:
# Separar los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Para la variable sexo los valores ahora son: 
Hombre : 1
Mujer : 2
Para la variable grupo_etnico los valores ahora son: 
Hombre : 1
Mujer : 2
Ignorado : 1
No indígena : 2
Indígena : 3
Mestizo, Ladino : 4
Maya : 5
Otro : 6
Xinca : 7
Garífuna : 8
Para la variable departamento los valores ahora son: 
Hombre : 1
Mujer : 2
Ignorado : 1
No indígena : 2
Indígena : 3
Mestizo, Ladino : 4
Maya : 5
Otro : 6
Xinca : 7
Garífuna : 8
Guatemala : 1
Zacapa : 2
Quetzaltenango : 3
Jutiapa : 4
El progreso : 5
Izabal : 6
Peten : 7
Huehuetenango : 8
Quiché : 9
Santa Rosa : 10
Jalapa : 11
Chimaltenango : 12
Escuintla : 13
San Marcos : 14
Sacatepequez : 15
Totonicapán : 16
Extranjero : 17
Retalhuleu : 18
Suchitepéquez : 19
Baja Verapaz : 20
Alta Verapaz : 21
Sololá : 22
Chiquimula : 23
Para la variable municipio los valores ahora son: 
Hombre : 1
Mujer : 2
Ignorado : 1
No indígena : 2
Indígena : 3
Mestizo, Ladino : 4
Maya : 5
Otro : 6
Xinca : 7
Garífuna : 8
Guatemala : 1
Zacapa : 2
Quetzaltenango : 3
Jutiapa 

In [21]:
regressionRandomForest(X_train, y_train, X_test, y_test)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score


def classificationRandomForest(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30]
    }

    # Crear el modelo de Random Forest
    rf_model = RandomForestClassifier(random_state=42)

    # Crear el objeto GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')

    # Entrenar el modelo
    grid_search.fit(X_train, y_train)

    # Obtener los mejores parámetros y el mejor estimador
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Entrenar el mejor modelo en todo el conjunto de entrenamiento
    best_estimator.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = best_estimator.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Precisión del modelo:", accuracy)

    # Calcular la precisión, el recall y el puntaje F1
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Precisión:", precision)
    print("Recall:", recall)
    print("Puntaje F1:", f1)

    # Calcular la matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=best_estimator.classes_, yticklabels=best_estimator.classes_)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

    return best_estimator, accuracy, precision, recall, f1